In [1]:
import os 
import pandas as pd 
import numpy as np
from scipy import stats
dataDir = r'C:\Users\jihhwan\sum_all_processing\total_raw(n=58)'
file_nms = os.listdir(dataDir)
file_nms_csv = [file for file in file_nms if file.endswith(".csv")]  
DF = pd.DataFrame(columns=['IN_IN', 'OUT_OUT'])


for fileName in file_nms_csv:
    total_dir = os.path.join(dataDir, fileName)
    
    csv_data = pd.read_csv(total_dir)
    case1 = csv_data.iloc[11:251:2,:6]
    case2 = csv_data.iloc[11:251:2,110]
    case3 = csv_data.iloc[11:251:2,111] 
    actual_output = pd.concat([case1, case2, case3], axis=1)
    actual_output.loc[(actual_output['par_left'] == 'B1')|(actual_output['par_left'] =='B2')|(actual_output['par_left'] =='B3'), 'par_Left']='B'
    actual_output.loc[(actual_output['par_left'] == 'A1')|(actual_output['par_left'] =='A2'), 'par_Left']='A'
    actual_output.loc[(actual_output['par_right'] == 'B1')|(actual_output['par_right'] =='B2')|(actual_output['par_right'] =='B3'), 'par_Right']='B'
    actual_output.loc[(actual_output['par_right'] == 'A1')|(actual_output['par_right'] =='A2'), 'par_Right']='A'
    #A, B translation

    actual_output.loc[actual_output['resource_level'] == 25.0, 'resource_level_1'] =3
    actual_output.loc[actual_output['resource_level'] == 20.0, 'resource_level_1'] =2
    actual_output.loc[actual_output['resource_level'] == 15.0, 'resource_level_1'] =1
    # 3 = 25, 2 = 20, 1 = 15

    actual_output.loc[actual_output['inequity_level_left'] == 6.0, 'inequity_level']=1
    actual_output.loc[actual_output['inequity_level_left'] == 8.0, 'inequity_level']=1
    actual_output.loc[actual_output['inequity_level_left'] == 10.0, 'inequity_level']=1
    actual_output.loc[actual_output['inequity_level_left'] == 3.0, 'inequity_level']=2
    actual_output.loc[actual_output['inequity_level_left'] == 4.0, 'inequity_level']=2
    actual_output.loc[actual_output['inequity_level_left'] == 5.0, 'inequity_level']=2
    # 1 = 6:4, 2 = 8:2 

    actual_output.loc[(actual_output['par_Right'] == 'A')&(actual_output['par_Left'] =='B'), 'who_adv']=1 #advantageous equity =1 
    actual_output.loc[(actual_output['par_Right'] == 'B')&(actual_output['par_Left'] =='A'), 'who_adv']=2  #disadcantageous equity = 2
    actual_output.loc[(actual_output['par_Right'] == 'A')&(actual_output['par_Left'] =='A'), 'who_adv']=3  #in_in
    actual_output.loc[(actual_output['par_Right'] == 'B')&(actual_output['par_Left'] =='B'), 'who_adv']=4  #out_out

    actual_output.loc[(actual_output['key_resp_2.keys'] == 'n'), 'response']=0
    actual_output.loc[(actual_output['key_resp_2.keys'] == 'y'), 'response']=1

    actual_output['reaction_time'] = actual_output['key_resp_2.rt']

    #print(actual_output.head())
    
    refined_data = actual_output.iloc[:,10:]
    csv_data = refined_data    #refining one data 
    refined_data["rt_z"] = 0
    
    for i in range(len(refined_data)):
        refined_data['rt_z'].iloc[i] = (refined_data['reaction_time'].iloc[i] - refined_data['reaction_time'].mean())/refined_data['reaction_time'].std()
    
    
    #expCondition = ['SMI','SMO','SMN','SEI','SEO','SEN','MMI','MMO','MMN','MEI','MEO','MEN','LMI','LMO','LMN','LEI','LEO','LEN']
    
    OUT_OUT = csv_data.loc[(csv_data['who_adv'] == 4)]['rt_z'].mean()
    IN_IN = csv_data.loc[(csv_data['who_adv'] == 3)]['rt_z'].mean()
    TOTAL = [IN_IN, OUT_OUT]
    TT = pd.DataFrame(TOTAL).T
    TT.columns = ('IN_IN', 'OUT_OUT')
    TT.index = [fileName[:-43]]
    DF = pd.concat([DF,TT])
DF.to_excel('ININ_OUTOUT_in_Neutral_zScore.xlsx')

C:\Users\jihhwan\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
